In [105]:
import pandas as pd
main_devs = pd.read_csv(r'C:\dev\repos\spring-petclinic\main_dev.csv')
main_devs.head()

,entity,main-dev,added,total-added,ownership
0,.bowerrc,IHABRITANE,3,4,0.75
1,.classpath,Costin Leau,43,81,0.53
2,.editorconfig,Antoine Rey,14,14,1.00
3,.gitignore,Mic,6,11,0.55
4,.mvn/wrapper/maven-wrapper.jar,patrickcrocker,0,0,0.00


In [106]:
initial_number_of_columns = len(main_devs.columns)
initial_number_of_columns

5

In [108]:
main_devs = main_devs.join(main_devs['entity'].str.split("/", expand=True))
main_devs.head()

,entity,main-dev,added,total-added,ownership,0,1,2,3,4,5,6,7,8,9,10
0,.bowerrc,IHABRITANE,3,4,0.75,.bowerrc,None,None,None,None,None,None,None,None,None,None
1,.classpath,Costin Leau,43,81,0.53,.classpath,None,None,None,None,None,None,None,None,None,None
2,.editorconfig,Antoine Rey,14,14,1.00,.editorconfig,None,None,None,None,None,None,None,None,None,None
3,.gitignore,Mic,6,11,0.55,.gitignore,None,None,None,None,None,None,None,None,None,None
4,.mvn/wrapper/maven-wrapper.jar,patrickcrocker,0,0,0.00,.mvn,wrapper,maven-wrapper.jar,None,None,None,None,None,None,None,None


In [109]:
columns_to_group = list(range(0, len(main_devs.columns)-initial_number_of_columns))
columns_to_group

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [121]:
hierarchical_main_devs = main_devs.fillna("").groupby(columns_to_group).first()
hierarchical_main_devs.head()

,,,,,,,,,,,entity,main-dev,added,total-added,ownership
0,1,2,3,4,5,6,7,8,9,10,,,,,
.bowerrc,,,,,,,,,,,.bowerrc,IHABRITANE,3,4,0.75
.classpath,,,,,,,,,,,.classpath,Costin Leau,43,81,0.53
.editorconfig,,,,,,,,,,,.editorconfig,Antoine Rey,14,14,1.00
.gitignore,,,,,,,,,,,.gitignore,Mic,6,11,0.55
.mvn,wrapper,maven-wrapper.jar,,,,,,,,,.mvn/wrapper/maven-wrapper.jar,patrickcrocker,0,0,0.00


In [122]:
hierarchical_main_devs = hierarchical_main_devs.replace("", pd.np.nan)
hierarchical_main_devs

entity  \
0                                      1                                                2                        3                       4               5       6          7                                                  8                                            9                                              10                                                                                         
.bowerrc                                                                                                                                                                                                                                                                                                                                                                                                   .bowerrc   
.classpath                                                                                                                                                                                                                                                                                                                                                                                               .classpath   
.editorconfig                                                                                                                                                                                                                                                                                                                                                                                         .editorconfig   
.gitignore                                                                                                                                                                                                                                                                                                                                                                                               .gitignore   
.mvn                                   wrapper                                          maven-wrapper.jar                                                                                                                                                                                                                                                                            .mvn/wrapper/maven-wrapper.jar   
                                                                                        maven-wrapper.properties                                                                                                                                                                                                                                                              .mvn/wrapper/maven-wrapper.properties   
.project                                                                                                                                                                                                                                                                                                                                                                                                   .project   
.settings                              .jsdtscope                                                                                                                                                                                                                                                                                                                                              .settings/.jsdtscope   
                                       com.springsource.server.ide.jdt.core.xml                                                                                                                                                                                                                                                        

In [114]:
import pandas as pd
import json


# order in the groupby here matters, it determines the json nesting
# the groupby call makes a pandas series by grouping 'the_parent' and 'the_child', while summing the numerical column 'child_size'
df1 = hierarchical_main_devs



# start a new flare.json document
flare = dict()
flare = {"name":"flare", "children": []}

d =dict()
d ['children'] = []

for line in df1.values:
    
    the_parent = line[0]
    the_child = line[1]
    child_size = line[2]

    # make a list of keys
    keys_list = []
    for item in d['children']:
        keys_list.append(item['name'])

    # if 'the_parent' is NOT a key in the flare.json yet, append it
    if not the_parent in keys_list:
        d['children'].append({"name":the_parent, "children":[{"name":the_child, "size":child_size}]})

    # if 'the_parent' IS a key in the flare.json, add a new child to it
    else:
        d['children'][keys_list.index(the_parent)]['children'].append({"name":the_child, "size":child_size})

flare = d


# export the final result to a json file
with open('data/flare.json', 'w') as outfile:
    json.dump(flare, outfile)

In [94]:
hierarchical_main_devs.reset_index().to_json("data/main_devs_split.json", orient='split')
hierarchical_main_devs.reset_index().to_json("data/main_devs_records.json", orient='records')
hierarchical_main_devs.reset_index().to_json("data/main_devs_index.json", orient='index')
hierarchical_main_devs.reset_index().to_json("data/main_devs_columns.json", orient='columns')
hierarchical_main_devs.reset_index().to_json("data/main_devs_values.json", orient='values')